In [1]:
library(ggplot2)
library(caTools)

In [2]:
source("/u/project/sriram/ukbiobank/33127/bd_load_data.R")
db<-data


Attaching package: ‘dplyr’

The following objects are masked from ‘package:data.table’:

    between, first, last

The following objects are masked from ‘package:stats’:

    filter, lag

The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



In [3]:
db$WBCC <- db$`30000-0.0`
db<- db %>% filter(!is.na(WBCC))

In [4]:
var<-c('31-0.0','1558-0.0','1239-0.0','1249-0.0','2100-0.0','21000-0.0')

In [5]:
db1<- db %>% select(var)

In [6]:
db1[which(db1$`21000-0.0`<1010 & db1$`21000-0.0`>1000),'21000-0.0']<-1
db1[which(db1$`21000-0.0`<2010 & db1$`21000-0.0`>2000),'21000-0.0']<-2
db1[which(db1$`21000-0.0`<3010 & db1$`21000-0.0`>3000),'21000-0.0']<-3
db1[which(db1$`21000-0.0`<4010 & db1$`21000-0.0`>4000),'21000-0.0']<-4
db1[which(db1$`21000-0.0`==5),'21000-0.0']<-3

In [7]:
db1 <- db1 %>% mutate_all(na_if,"") 
db1<- db1 %>% mutate_each(funs(replace(., .<0, NA)))
db1<- db1 %>% mutate_all(funs(factor(.)))
db1$eid<-db$eid
db1$WBCC <- db$WBCC
db1<- db1 %>% select(c('eid','WBCC'),everything())

Warning message:
“funs() is soft deprecated as of dplyr 0.8.0
Please use a list of either functions or lambdas: 

  # Simple named list: 
  list(mean = mean, median = median)

  # Auto named with `tibble::lst()`: 
  tibble::lst(mean, median)

  # Using lambdas
  list(~ mean(., trim = .2), ~ median(., na.rm = TRUE))
This warning is displayed once per session.”

In [ ]:
disease<-read.csv('more_feature.csv')

In [ ]:
dim(d)

In [9]:
disease1<- disease[,-1] %>% mutate_all(funs(factor(.)))
disease1$eid<-disease$eid
disease1<- disease1 %>% select('eid',everything())

In [10]:
df<-merge(db1,disease1,by='eid')

In [11]:
df1<- df%>% select(-c('eid','WBCC')) %>% mutate_all(funs(droplevels(.)))

In [12]:
df1$eid<-df$eid
df1$WBCC<-df$WBCC
df1<-na.omit(df1)

In [13]:
head(df1)

31-0.0,1558-0.0,1239-0.0,1249-0.0,2100-0.0,21000-0.0,Abdominal.hernia,Anemias,bacterial.infection,Benign.and.malignant.neoplasm.of.colon,⋯,Disorders.of.lipoid.metabolism,DISORDERS.OF.THYROID.GLAND,esophageal.disorders,HYPERTENSIVE.DISEASE,ISCHEMIC.HEART.DISEASE,Malignant.neoplasm.of.skin,mononeuritis.and.nerve.lesions,noninfective.gastrointestinal.disorders,eid,WBCC
<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,⋯,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<fct>,<int>,<dbl>
1,1,0,1,0,1,0,0,0,0,⋯,0,0,0,0,0,0,0,0,1000032,4.98
0,1,2,2,0,1,0,0,0,1,⋯,0,0,0,0,0,0,1,0,1000060,8.10
1,2,0,3,0,1,0,0,0,0,⋯,0,0,0,0,1,0,0,0,1000102,7.81
1,1,0,4,0,1,0,0,0,1,⋯,0,0,0,0,0,0,0,0,1000116,2.92
1,6,0,1,0,1,0,0,0,0,⋯,0,0,0,0,1,0,0,0,1000121,6.04
1,2,0,1,0,1,0,0,1,0,⋯,0,0,0,0,0,0,0,0,1000195,6.50


In [14]:
dim(df1)

[1] 193642     24

In [15]:
set.seed(1233) 
sample = sample.split(df1,SplitRatio = 0.7) 
train1 =subset(df1,sample ==TRUE)
test1=subset(df1, sample==FALSE)

In [16]:
model1<-aov(WBCC~.  -eid -Cataract -Malignant.neoplasm.of.skin -mononeuritis.and.nerve.lesions -Anemias-DISORDERS.OF.THYROID.GLAND-`2100-0.0`-Benign.and.malignant.neoplasm.of.colon, data =train1)

In [17]:
summary(model1)

                                                      Df Sum Sq Mean Sq F value
`31-0.0`                                               1    499   498.8  127.43
`1558-0.0`                                             5   2866   573.2  146.45
`1239-0.0`                                             1    731   731.1  186.79
`1249-0.0`                                             3   2533   844.3  215.70
`21000-0.0`                                            4   2552   637.9  162.97
Abdominal.hernia                                       1    297   297.2   75.92
bacterial.infection                                    1    769   769.1  196.49
cancer.of.female.genital.organs..breast.dysplasia      1    177   177.3   45.29
DIABETES                                               1    765   764.9  195.43
diseases.of.veins.and.lymphatics                       1    598   598.5  152.90
Disorders.of.lipoid.metabolism                         1     96    95.8   24.47
esophageal.disorders                    

In [18]:
hh<-predict(model1, test1)

In [19]:
SSR<-sum((hh-mean(test1$WBCC))^2)
SST<-sum((test1$WBCC-mean(test1$WBCC))^2)
SSR/SST

[1] 0.0133807

In [21]:
model_v1<- aov(WBCC~`31-0.0`,data=train1)
summary(model_v1)

                Df Sum Sq Mean Sq F value Pr(>F)    
`31-0.0`         1    499   498.8   124.5 <2e-16 ***
Residuals   129094 517039     4.0                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [22]:
model_v2<- aov(WBCC~`1558-0.0`,data=train1)
summary(model_v2)

                Df Sum Sq Mean Sq F value Pr(>F)    
`1558-0.0`       5   2342   468.3   117.3 <2e-16 ***
Residuals   129090 515196     4.0                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [23]:
model_v3<- aov(WBCC~`1239-0.0` ,data=train1)
summary(model_v3)

                Df Sum Sq Mean Sq F value Pr(>F)    
`1239-0.0`       1    740     740   184.8 <2e-16 ***
Residuals   129094 516797       4                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [24]:
model_v4<- aov(WBCC~`1249-0.0` ,data=train1)
summary(model_v4)

                Df Sum Sq Mean Sq F value Pr(>F)    
`1249-0.0`       3   2417   805.7   201.9 <2e-16 ***
Residuals   129092 515120     4.0                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [25]:
model_v5<- aov(WBCC~`21000-0.0`,data=train1)
summary(model_v5)

                Df Sum Sq Mean Sq F value Pr(>F)    
`21000-0.0`      4   2378   594.4   148.9 <2e-16 ***
Residuals   129091 515160     4.0                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [26]:
model_v6<- aov(WBCC~Abdominal.hernia  ,data=train1)
summary(model_v6)

                     Df Sum Sq Mean Sq F value Pr(>F)    
Abdominal.hernia      1    287  286.85   71.59 <2e-16 ***
Residuals        129094 517250    4.01                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [27]:
model_v7<- aov(WBCC~bacterial.infection  ,data=train1)
summary(model_v7)

                        Df Sum Sq Mean Sq F value Pr(>F)    
bacterial.infection      1   1122    1122   280.4 <2e-16 ***
Residuals           129094 516415       4                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [28]:
model_v8<- aov(WBCC~cancer.of.female.genital.organs..breast.dysplasia  ,data=train1)
summary(model_v8)

                                                      Df Sum Sq Mean Sq F value
cancer.of.female.genital.organs..breast.dysplasia      1    361     361    90.1
Residuals                                         129094 517176       4        
                                                  Pr(>F)    
cancer.of.female.genital.organs..breast.dysplasia <2e-16 ***
Residuals                                                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [29]:
model_v9<- aov(WBCC~DIABETES ,data=train1)
summary(model_v9)

                Df Sum Sq Mean Sq F value Pr(>F)    
DIABETES         1   1528    1528   382.2 <2e-16 ***
Residuals   129094 516009       4                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [30]:
model_v10<- aov(WBCC~diseases.of.veins.and.lymphatics,data=train1)
summary(model_v10)

                                     Df Sum Sq Mean Sq F value Pr(>F)    
diseases.of.veins.and.lymphatics      1    784   783.8   195.8 <2e-16 ***
Residuals                        129094 516754     4.0                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [31]:
model_v11<- aov(WBCC~Disorders.of.lipoid.metabolism,data=train1)
summary(model_v11)

                                   Df Sum Sq Mean Sq F value   Pr(>F)    
Disorders.of.lipoid.metabolism      1     81   80.79   20.16 7.15e-06 ***
Residuals                      129094 517457    4.01                     
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [32]:
model_v12<- aov(WBCC~esophageal.disorders,data=train1)
summary(model_v12)

                         Df Sum Sq Mean Sq F value   Pr(>F)    
esophageal.disorders      1    109  108.66   27.11 1.92e-07 ***
Residuals            129094 517429    4.01                     
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [33]:
model_v13<- aov(WBCC~HYPERTENSIVE.DISEASE,data=train1)
summary(model_v13)

                         Df Sum Sq Mean Sq F value   Pr(>F)    
HYPERTENSIVE.DISEASE      1     77   77.23   19.27 1.14e-05 ***
Residuals            129094 517460    4.01                     
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [35]:
model_v14<- aov(WBCC~ISCHEMIC.HEART.DISEASE,data=train1)
summary(model_v14)

                           Df Sum Sq Mean Sq F value Pr(>F)    
ISCHEMIC.HEART.DISEASE      1    277  277.29    69.2 <2e-16 ***
Residuals              129094 517260    4.01                   
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

In [37]:
model_v15<- aov(WBCC~noninfective.gastrointestinal.disorders,data=train1)
summary(model_v15)

                                            Df Sum Sq Mean Sq F value   Pr(>F)
noninfective.gastrointestinal.disorders      1    146  146.20   36.48 1.55e-09
Residuals                               129094 517391    4.01                 
                                           
noninfective.gastrointestinal.disorders ***
Residuals                                  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1